In [1]:
import ray
import random

# Inicjalizacja Ray
ray.init()

# Definicja aktora NameNode
@ray.remote
class NameNode:
    def __init__(self):
        # Słownik przechowujący mapowanie nazwy artefaktu do listy bloków
        self.artifact_blocks = {}

    def distribute_artifact(self, artifact_name, artifact_data, num_copies=3, block_size=100):
        # Obliczanie liczby bloków
        num_blocks = (len(artifact_data) + block_size - 1) // block_size

        # Generowanie kopii bloków
        blocks = [[random.randint(1, 100) for _ in range(block_size)] for _ in range(num_blocks)]
        
        # Przypisanie bloków do artefaktu
        self.artifact_blocks[artifact_name] = blocks

        # Zwrócenie informacji o przypisaniu bloków
        return {"artifact_name": artifact_name, "blocks": blocks}


# Definicja aktora StorageNode
@ray.remote
class StorageNode:
    def __init__(self, node_id):
        self.node_id = node_id
        self.stored_blocks = {}

    def store_blocks(self, blocks):
        # Przechowywanie bloków na węźle
        for block_index, block_data in enumerate(blocks):
            self.stored_blocks[block_index] = block_data

        # Zwrócenie informacji o przechowywanych blokach
        return {"node_id": self.node_id, "stored_blocks": list(self.stored_blocks.keys())}


# Inicjalizacja aktora NameNode
name_node = NameNode.remote()

# Inicjalizacja aktorów StorageNode
storage_nodes = [StorageNode.remote(node_id) for node_id in range(1, 4)]

# Dystrybucja artefaktu przez NameNode
artifact_name = "example_artifact"
artifact_data = "Lorem ipsum dolor sit amet, consectetur adipiscing elit."
result = ray.get(name_node.distribute_artifact.remote(artifact_name, artifact_data))

# Przechowywanie bloków przez StorageNodes
for storage_node in storage_nodes:
    ray.get(storage_node.store_blocks.remote(result["blocks"]))

# Wyświetlenie przechowywanych bloków na StorageNodes
for storage_node in storage_nodes:
    print("Storage Node ID:", storage_node.node_id)
    print("Stored Blocks:", ray.get(storage_node.stored_blocks.remote()))
    print()


2024-04-18 19:37:20,448	INFO worker.py:1743 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


AttributeError: 'ActorHandle' object has no attribute 'node_id'